In [68]:
import tensorflow as tf
print("imported")

Hello
imported


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

#From the above observation we come to know that there is no imbalance in our data

In [9]:
df.replace({"sentiment":{"positive":1,"negative":0}},  inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [34]:
#Splitting data into training data and test data
train_data,test_data = train_test_split(df, test_size=0.2, random_state=42)

In [35]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


In [36]:
#Preprocessing the data
tokenizer= Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [37]:
print(X_train)
print(X_test)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [38]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

In [39]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [58]:
import numpy as np

model = Sequential([
    Embedding(input_dim=5000, output_dim=128),
    LSTM(128),
    Dense(1, activation='sigmoid')
])

# Provide dummy input data
dummy_input = np.zeros((1, 200))  # Batch size 1, sequence length 200
model(dummy_input)
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)             │ (1, 200, 128)               │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ (1, 128)                    │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (1, 1)                      │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
#Compiling the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [60]:
#Training the model
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 153s 300ms/step - accuracy: 0.7304 - loss: 0.5205 - val_accuracy: 0.8701 - val_loss: 0.3148
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 202s 404ms/step - accuracy: 0.8863 - loss: 0.2853 - val_accuracy: 0.8706 - val_loss: 0.2995
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 402ms/step - accuracy: 0.9170 - loss: 0.2201 - val_accuracy: 0.8835 - val_loss: 0.2979
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 389ms/step - accuracy: 0.9410 - loss: 0.1628 - val_accuracy: 0.8800 - val_loss: 0.3208
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 209s 419ms/step - accuracy: 0.9523 - loss: 0.1362 - val_accuracy: 0.8795 - val_loss: 0.3318


In [62]:
#Model Evaluation
results = model.evaluate(X_test, Y_test, verbose=1)

print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 87ms/step - accuracy: 0.8769 - loss: 0.3170
Test Loss: 0.3129194974899292
Test Accuracy: 0.8799999952316284


In [70]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

In [71]:

classifier={
              "Logistic Regression":LogisticRegression(),
               "Decision Tree Classifier":DecisionTreeClassifier()
}
for name, clf in classifier.items():
    print(f"\n========{name}======")
    clf.fit(X_train,Y_train)
    Y_pred=clf.predict(X_test)
    print(f"\n Accuracy:{accuracy_score(Y_test,Y_pred)}")
    print(f"\n Precision:{precision_score(Y_test,Y_pred)}")
    print(f"\n Recall:{recall_score(Y_test,Y_pred)}")
    print(f"\n F1 score:{f1_score(Y_test,Y_pred)}")


========Logistic Regression======

 Accuracy:0.5131

 Precision:0.5186077057793345

 Recall:0.4701329628894622

 F1 score:0.49318205475174354

========Decision Tree Classifier======

 Accuracy:0.5085

 Precision:0.5121425773599687

 Recall:0.5189521730502084

 F1 score:0.5155248891079349


In [72]:
#Predictive System
def predict_sentiment(review):
    sequence=tokenizer.text_to_sequence(review)
    padded_sequence=pad_sequence(sequence,maxlen=200)
    prediction=model.predict(padded_sequence)
    sentiment="positive"if prediction[0][0]>0.5 else "negative"
    return sentiment

In [75]:
# Example review
review = "The movie was fantastic! Absolutely loved it."

# Predict sentiment
sentiment = predict_sentiment(review)
print(f"Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Sentiment: positive
